# Data Generator 

En este archivo estara la script encargada de generar informacion y de meterla en SQL de forma sencilla


In [1]:
import sqlite3


In [2]:
# Conexion


In [ ]:
# Generacion de Data
def generar_Info(categoria, datos, sizes):
    data = []
    for i in categoria:
        tempData =[]
        